In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
from scipy.spatial.distance import mahalanobis as mah_dist # u, v, cov_mat
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))


def po_linear_model(graph, alpha=1, beta=1, sigma=0.1, hete=0.5, gamma=2):    
    hete = 0.5
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i]  

def po_multiplicative_model(graph, alpha=1, sigma=0.1, delta=1, gamma=2): 
    for i in graph.nodes:
        graph.nodes[i]["y"] = ( (alpha + sigma * np.random.normal()) * graph.degree[i]/avg_deg )  *  (1 + delta * graph.nodes[i]["z"] + gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]]) / len(graph[i]) )




DGL backend not selected or invalid.  Assuming PyTorch for now.
/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["n_cl"] = len(node_to_connected_clusters[i])
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)

        # self.conv1 = dgl.nn.GINConv(nn.Linear(3,16), "mean")
        # self.conv2 = dgl.nn.GINConv(nn.Linear(16,8), "mean")
        # self.conv3 = dgl.nn.GINConv(nn.Linear(8,1), "mean")

        # self.conv1 = dgl.nn.GraphConv(3, 8)
        # self.conv2 = dgl.nn.GraphConv(8, 8)
        # self.conv3 = dgl.nn.GraphConv(8, 1)
        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [6]:
# for num_step in range(num_step):
#     # p_list = [0, ramps[num_step]]
#     # print(p_list)
#     print(ramps[num_step])

In [ ]:
# cluster-level cr

GATE_hat_list = np.zeros((repeat_num, len(ramps)))

     
for seed in tqdm(range(repeat_num)):
    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_cluster,))   
    
    for num_step in range(len(ramps)): 
        torch.manual_seed(2)
        p_list = [0, ramps[num_step]]
    
        print(f"Seed: {seed}")
        
        g_feat_list = []
        y_list = []
        
        for p in p_list:
            
            tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]
            nx.set_node_attributes(g, 0, "z")

            if len(tr_clusters) > 0:
                tr_units = reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters])                  
                nx.set_node_attributes(g, {unit:1 for unit in tr_units}, "z")
            
            # po_linear_model(g, gamma = 1) 
            po_multiplicative_model(g, gamma = 1)
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()
    
    
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
        
    
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    # torch.save(GATE_hat_list, "Result/2step_CR_cluster.pkl")
    torch.save(GATE_hat_list, "Result/Multiplicative_incre_2step_CR_cluster.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.00, Loss: 4.5867
Epoch 0, Treat Prop: 0.02, Loss: 0.6727
Epoch 50, Treat Prop: 0.00, Loss: 0.0229
Epoch 50, Treat Prop: 0.02, Loss: 0.0492
Epoch 100, Treat Prop: 0.00, Loss: 0.0207
Epoch 100, Treat Prop: 0.02, Loss: 0.0453
Epoch 150, Treat Prop: 0.00, Loss: 0.0199
Epoch 150, Treat Prop: 0.02, Loss: 0.0430
Epoch 200, Treat Prop: 0.00, Loss: 0.0197
Epoch 200, Treat Prop: 0.02, Loss: 0.0412
Epoch 250, Treat Prop: 0.00, Loss: 0.0197
Epoch 250, Treat Prop: 0.02, Loss: 0.0395
Epoch 300, Treat Prop: 0.00, Loss: 0.0196
Epoch 300, Treat Prop: 0.02, Loss: 0.0378
Epoch 350, Treat Prop: 0.00, Loss: 0.0196
Epoch 350, Treat Prop: 0.02, Loss: 0.0361
0.034390900284051895
Seed: 0
Epoch 0, Treat Prop: 0.00, Loss: 4.5597
Epoch 0, Treat Prop: 0.05, Loss: 0.6633
Epoch 50, Treat Prop: 0.00, Loss: 0.0259
Epoch 50, Treat Prop: 0.05, Loss: 0.1173
Epoch 100, Treat Prop: 0.00, Loss: 0.0238
Epoch 100, Treat Prop: 0.05, Loss: 0.1101
Epoch 150, Treat Prop: 0.00, Loss: 0.0232
Epoch 150, Treat 

  0%|          | 1/1000 [00:20<5:46:33, 20.81s/it]

...

0.2682175040245056
Seed: 370
Epoch 0, Treat Prop: 0.00, Loss: 4.5432
Epoch 0, Treat Prop: 0.02, Loss: 0.6384
Epoch 50, Treat Prop: 0.00, Loss: 0.0225
Epoch 50, Treat Prop: 0.02, Loss: 0.0572
Epoch 100, Treat Prop: 0.00, Loss: 0.0208
Epoch 100, Treat Prop: 0.02, Loss: 0.0535
Epoch 150, Treat Prop: 0.00, Loss: 0.0202
Epoch 150, Treat Prop: 0.02, Loss: 0.0515
Epoch 200, Treat Prop: 0.00, Loss: 0.0200
Epoch 200, Treat Prop: 0.02, Loss: 0.0497
Epoch 250, Treat Prop: 0.00, Loss: 0.0200
Epoch 250, Treat Prop: 0.02, Loss: 0.0482
Epoch 300, Treat Prop: 0.00, Loss: 0.0200
Epoch 300, Treat Prop: 0.02, Loss: 0.0468
Epoch 350, Treat Prop: 0.00, Loss: 0.0200
Epoch 350, Treat Prop: 0.02, Loss: 0.0454
0.04416259005665779
Seed: 370
Epoch 0, Treat Prop: 0.00, Loss: 4.5526
Epoch 0, Treat Prop: 0.05, Loss: 0.6804
Epoch 50, Treat Prop: 0.00, Loss: 0.0311
Epoch 50, Treat Prop: 0.05, Loss: 0.1595
Epoch 100, Treat Prop: 0.00, Loss: 0.0286
Epoch 100, Treat Prop: 0.05, Loss: 0.1511
Epoch 150, Treat Prop: 0.00, 

 37%|███▋      | 371/1000 [2:06:18<3:45:01, 21.46s/it]

0.1758861392736435
Seed: 371
Epoch 0, Treat Prop: 0.00, Loss: 4.5451
Epoch 0, Treat Prop: 0.02, Loss: 0.6585
Epoch 50, Treat Prop: 0.00, Loss: 0.0229
Epoch 50, Treat Prop: 0.02, Loss: 0.0771
Epoch 100, Treat Prop: 0.00, Loss: 0.0214
Epoch 100, Treat Prop: 0.02, Loss: 0.0729
Epoch 150, Treat Prop: 0.00, Loss: 0.0209
Epoch 150, Treat Prop: 0.02, Loss: 0.0700
Epoch 200, Treat Prop: 0.00, Loss: 0.0207
Epoch 200, Treat Prop: 0.02, Loss: 0.0674
Epoch 250, Treat Prop: 0.00, Loss: 0.0207
Epoch 250, Treat Prop: 0.02, Loss: 0.0646
Epoch 300, Treat Prop: 0.00, Loss: 0.0208
Epoch 300, Treat Prop: 0.02, Loss: 0.0613
Epoch 350, Treat Prop: 0.00, Loss: 0.0209
Epoch 350, Treat Prop: 0.02, Loss: 0.0576
0.05367136001586914
Seed: 371
Epoch 0, Treat Prop: 0.00, Loss: 4.5807
Epoch 0, Treat Prop: 0.05, Loss: 0.7077
Epoch 50, Treat Prop: 0.00, Loss: 0.0305
Epoch 50, Treat Prop: 0.05, Loss: 0.1757
Epoch 100, Treat Prop: 0.00, Loss: 0.0250
Epoch 100, Treat Prop: 0.05, Loss: 0.1713
Epoch 150, Treat Prop: 0.00, 

 37%|███▋      | 372/1000 [2:06:38<3:41:16, 21.14s/it]

0.20706407725811005
Seed: 372
Epoch 0, Treat Prop: 0.00, Loss: 4.5396
Epoch 0, Treat Prop: 0.02, Loss: 0.6852
Epoch 50, Treat Prop: 0.00, Loss: 0.0242
Epoch 50, Treat Prop: 0.02, Loss: 0.0812
Epoch 100, Treat Prop: 0.00, Loss: 0.0227
Epoch 100, Treat Prop: 0.02, Loss: 0.0762
Epoch 150, Treat Prop: 0.00, Loss: 0.0223
Epoch 150, Treat Prop: 0.02, Loss: 0.0723
Epoch 200, Treat Prop: 0.00, Loss: 0.0221
Epoch 200, Treat Prop: 0.02, Loss: 0.0686
Epoch 250, Treat Prop: 0.00, Loss: 0.0221
Epoch 250, Treat Prop: 0.02, Loss: 0.0646
Epoch 300, Treat Prop: 0.00, Loss: 0.0221
Epoch 300, Treat Prop: 0.02, Loss: 0.0601
Epoch 350, Treat Prop: 0.00, Loss: 0.0222
Epoch 350, Treat Prop: 0.02, Loss: 0.0552
0.05046570673584938
Seed: 372
Epoch 0, Treat Prop: 0.00, Loss: 4.5593
Epoch 0, Treat Prop: 0.05, Loss: 0.7003
Epoch 50, Treat Prop: 0.00, Loss: 0.0281
Epoch 50, Treat Prop: 0.05, Loss: 0.1417
Epoch 100, Treat Prop: 0.00, Loss: 0.0253
Epoch 100, Treat Prop: 0.05, Loss: 0.1321


array([[0.55846262, 1.08712709, 1.62418032, 1.8334074 , 1.92653811],
       [0.69687438, 0.98067021, 1.24898303, 1.76001835, 1.8925755 ],
       [0.62316918, 0.90868449, 1.4823252 , 1.76998889, 1.90446496],
       ...,
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])